In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [8]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import ADASYN
from sklearn.cluster import KMeans

cc= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))
ada= ADASYN(random_state= 15)

X_cc, y_cc= cc.fit_resample(X_train, y_train)
X_cc_ada, y_cc_ada= ada.fit_resample(X_cc, y_cc)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_cc_ada= LogisticRegression(random_state=15)
lr_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_lr_cc_ada= lr_cc_ada.predict(X_test)
y_pred_prob_lr_cc_ada= lr_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_cc_ada, labels= lr_cc_ada.classes_)
print(classification_report(y_test, y_pred_lr_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_lr_cc_ada))

              precision    recall  f1-score   support

           0       0.97      0.71      0.82      8517
           1       0.23      0.80      0.36       912

    accuracy                           0.72      9429
   macro avg       0.60      0.76      0.59      9429
weighted avg       0.90      0.72      0.78      9429

Precision: 0.2316919191919192
Accuracy: 0.7229822886838477
AUC: 0.7595215914919389


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_cc_ada= DecisionTreeClassifier(random_state=15)
dt_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_dt_cc_ada= dt_cc_ada.predict(X_test)
y_pred_prob_dt_cc_ada= dt_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_cc_ada, labels= dt_cc_ada.classes_)
print(classification_report(y_test, y_pred_dt_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_cc_ada))

              precision    recall  f1-score   support

           0       0.96      0.14      0.24      8517
           1       0.10      0.94      0.19       912

    accuracy                           0.22      9429
   macro avg       0.53      0.54      0.21      9429
weighted avg       0.88      0.22      0.24      9429

Precision: 0.10491044230534909
Accuracy: 0.21550535581715982
AUC: 0.5407843368989574


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_cc_ada= GridSearchCV(estimator= dt_cc_ada, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_cc_ada.fit(X_cc_ada, y_cc_ada)

print("Best Parameters:", gs_dt_cc_ada.best_params_)
print("Best Precision Score:", gs_dt_cc_ada.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best Precision Score: 0.9334973930435556


In [12]:
tuned_dt_cc_ada= gs_dt_cc_ada.best_estimator_
y_pred_tuned_dt_cc_ada= tuned_dt_cc_ada.predict(X_test)
y_pred_prob_tuned_dt_cc_ada= tuned_dt_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_cc_ada, labels= tuned_dt_cc_ada.classes_)
print(classification_report(y_test, y_pred_tuned_dt_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_cc_ada))

              precision    recall  f1-score   support

           0       0.95      0.11      0.20      8517
           1       0.10      0.95      0.19       912

    accuracy                           0.20      9429
   macro avg       0.53      0.53      0.20      9429
weighted avg       0.87      0.20      0.20      9429

Precision: 0.10300939693112882
Accuracy: 0.19535475660197263
AUC: 0.5415264028187176


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_cc_ada= KNeighborsClassifier()
knn_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_knn_cc_ada= knn_cc_ada.predict(X_test)
y_pred_prob_knn_cc_ada= knn_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_cc_ada, labels= knn_cc_ada.classes_)
print(classification_report(y_test, y_pred_knn_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_cc_ada))

              precision    recall  f1-score   support

           0       0.93      0.84      0.88      8517
           1       0.22      0.44      0.30       912

    accuracy                           0.80      9429
   macro avg       0.58      0.64      0.59      9429
weighted avg       0.86      0.80      0.83      9429

Precision: 0.22422535211267605
Accuracy: 0.799448509916216
AUC: 0.736063862986102


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_cc_ada= GridSearchCV(estimator= knn_cc_ada, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_cc_ada.fit(X_cc_ada, y_cc_ada)
print("Best Parameters:", gs_knn_cc_ada.best_params_)
print("Best Precision Score:", gs_knn_cc_ada.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 12, 'weights': 'uniform'}
Best Precision Score: 0.8322459511755268


In [15]:
tuned_knn_cc_ada= gs_knn_cc_ada.best_estimator_
y_pred_tuned_knn_cc_ada= tuned_knn_cc_ada.predict(X_test)
y_pred_prob_tuned_knn_cc_ada= tuned_knn_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_cc_ada, labels= tuned_knn_cc_ada.classes_)
print(classification_report(y_test, y_pred_tuned_knn_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_cc_ada))

              precision    recall  f1-score   support

           0       0.98      0.57      0.72      8517
           1       0.18      0.88      0.30       912

    accuracy                           0.60      9429
   macro avg       0.58      0.73      0.51      9429
weighted avg       0.90      0.60      0.68      9429

Precision: 0.18124578367438723
Accuracy: 0.60260897231944
AUC: 0.798561481268629


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_cc_ada= GaussianNB()
nb_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_nb_cc_ada= nb_cc_ada.predict(X_test)
y_pred_prob_nb_cc_ada= nb_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_cc_ada, labels= nb_cc_ada.classes_)
print(classification_report(y_test, y_pred_nb_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_cc_ada))

              precision    recall  f1-score   support

           0       0.98      0.39      0.56      8517
           1       0.14      0.91      0.24       912

    accuracy                           0.44      9429
   macro avg       0.56      0.65      0.40      9429
weighted avg       0.90      0.44      0.53      9429

Precision: 0.13855622089155023
Accuracy: 0.44235868066603035
AUC: 0.7940834018238034


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_cc_ada= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_svc_cc_ada= svc_cc_ada.predict(X_test)
y_pred_prob_svc_cc_ada= svc_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_cc_ada, labels= svc_cc_ada.classes_)
print(classification_report(y_test, y_pred_svc_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_cc_ada))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93      8517
           1       0.13      0.07      0.09       912

    accuracy                           0.86      9429
   macro avg       0.52      0.51      0.51      9429
weighted avg       0.83      0.86      0.85      9429

Precision: 0.1267056530214425
Accuracy: 0.8626577579806979
AUC: 0.5886960598925988


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_cc_ada= RandomForestClassifier(random_state=15)
rf_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_rf_cc_ada= rf_cc_ada.predict(X_test)
y_pred_prob_rf_cc_ada= rf_cc_ada.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_cc_ada, labels= rf_cc_ada.classes_)
print(classification_report(y_test, y_pred_rf_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_cc_ada))

              precision    recall  f1-score   support

           0       0.99      0.05      0.10      8517
           1       0.10      1.00      0.18       912

    accuracy                           0.14      9429
   macro avg       0.55      0.52      0.14      9429
weighted avg       0.91      0.14      0.11      9429

Precision: 0.10107861670187925
Accuracy: 0.1423268639304274
AUC: 0.6501648405974428


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_cc_ada= GridSearchCV(estimator= rf_cc_ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_cc_ada.fit(X_cc_ada, y_cc_ada)
print("Best Parameters:", gs_tuned_rf_cc_ada.best_params_)
print("Best Precision Score:", gs_tuned_rf_cc_ada.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best Precision Score: 0.9402407277028054


In [21]:
tuned_rf_cc_ada= gs_tuned_rf_cc_ada.best_estimator_
y_pred_tuned_rf_cc_ada= tuned_rf_cc_ada.predict(X_test)
y_pred_prob_tuned_rf_cc_ada= tuned_rf_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_cc_ada, labels= tuned_rf_cc_ada.classes_)
print(classification_report(y_test, y_pred_tuned_rf_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_cc_ada))

              precision    recall  f1-score   support

           0       0.99      0.06      0.11      8517
           1       0.10      1.00      0.18       912

    accuracy                           0.15      9429
   macro avg       0.55      0.53      0.15      9429
weighted avg       0.91      0.15      0.12      9429

Precision: 0.10183938986092418
Accuracy: 0.1502810478311592
AUC: 0.6525974753279818


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_cc_ada= AdaBoostClassifier(random_state=15)
ada_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_ada_cc_ada= ada_cc_ada.predict(X_test)
y_pred_prob_ada_cc_ada= ada_cc_ada.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_cc_ada, labels= ada_cc_ada.classes_)
print(classification_report(y_test, y_pred_ada_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_cc_ada))

              precision    recall  f1-score   support

           0       0.96      0.04      0.08      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.13      9429
weighted avg       0.87      0.13      0.09      9429

Precision: 0.09903729113643908
Accuracy: 0.1346908473857249
AUC: 0.6656724605484594


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_cc_ada= GridSearchCV(estimator= ada_cc_ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_cc_ada.fit(X_cc_ada, y_cc_ada)
print("Best Parameters:", gs_ada_cc_ada.best_params_)
print("Best Precision Score:", gs_ada_cc_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9581801662096966


In [25]:
tuned_ada_cc_ada= gs_ada_cc_ada.best_estimator_
y_pred_tuned_ada_cc_ada= tuned_ada_cc_ada.predict(X_test)
y_pred_prob_tuned_ada_cc_ada= tuned_ada_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_cc_ada, labels= tuned_ada_cc_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_cc_ada))

              precision    recall  f1-score   support

           0       0.95      0.03      0.05      8517
           1       0.10      0.99      0.18       912

    accuracy                           0.12      9429
   macro avg       0.52      0.51      0.12      9429
weighted avg       0.87      0.12      0.07      9429

Precision: 0.09790895229797429
Accuracy: 0.1201612047937215
AUC: 0.6332946207687824


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_cc_ada= GradientBoostingClassifier(random_state=15)
grb_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_grb_cc_ada= grb_cc_ada.predict(X_test)
y_pred_prob_grb_cc_ada= grb_cc_ada.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_cc_ada, labels= grb_cc_ada.classes_)
print(classification_report(y_test, y_pred_grb_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_cc_ada))

              precision    recall  f1-score   support

           0       0.97      0.05      0.09      8517
           1       0.10      0.99      0.18       912

    accuracy                           0.14      9429
   macro avg       0.54      0.52      0.13      9429
weighted avg       0.89      0.14      0.10      9429

Precision: 0.09981167608286252
Accuracy: 0.13702407466327288
AUC: 0.6629925134251621


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_cc_ada= GridSearchCV(estimator= grb_cc_ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_cc_ada.fit(X_cc_ada, y_cc_ada)

print("Best Parameters:", gs_grb_cc_ada.best_params_)
print("Best Precision Score:", gs_grb_cc_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.9559080217228688


In [28]:
tuned_grb_cc_ada= gs_grb_cc_ada.best_estimator_
y_pred_tuned_grb_cc_ada= tuned_grb_cc_ada.predict(X_test)
y_pred_prob_tuned_grb_cc_ada= tuned_grb_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_cc_ada, labels= tuned_grb_cc_ada.classes_)
print(classification_report(y_test, y_pred_tuned_grb_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_cc_ada))

              precision    recall  f1-score   support

           0       0.96      0.04      0.07      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.13      9429
weighted avg       0.87      0.13      0.08      9429

Precision: 0.09869216397406308
Accuracy: 0.12875172340651184
AUC: 0.6516565681845802


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_cc_ada= XGBClassifier(random_state=15)
xgb_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_xgb_cc_ada= xgb_cc_ada.predict(X_test)
y_pred_prob_xgb_cc_ada= xgb_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_cc_ada, labels= xgb_cc_ada.classes_)
print(classification_report(y_test, y_pred_xgb_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_cc_ada))

              precision    recall  f1-score   support

           0       0.94      0.04      0.07      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.52      0.51      0.13      9429
weighted avg       0.86      0.13      0.08      9429

Precision: 0.0981169474727453
Accuracy: 0.1291759465478842
AUC: 0.6268639192203828


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_cc_ada= GridSearchCV(estimator= xgb_cc_ada, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_cc_ada.fit(X_cc_ada, y_cc_ada)

print("Best Parameters:", gs_xgb_cc_ada.best_params_)
print("Best Precision Score:", gs_xgb_cc_ada.best_score_)

tuned_xgb_cc_ada= gs_xgb_cc_ada.best_estimator_
y_pred_tuned_xgb_cc_ada= tuned_xgb_cc_ada.predict(X_test)
y_pred_prob_tuned_xgb_cc_ada= tuned_xgb_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_cc_ada, labels= tuned_xgb_cc_ada.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_cc_ada))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 1.0}
Best Precision Score: 0.9544346759304823
              precision    recall  f1-score   support

           0       0.95      0.04      0.08      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.52      0.51      0.13      9429
weighted avg       0.87      0.13      0.09      9429

Precision: 0.09855264611645122
Accuracy: 0.13256973167886307
AUC: 0.6096273654960461


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_cc_ada= LGBMClassifier(random_state=15)
lgm_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_lgm_cc_ada= lgm_cc_ada.predict(X_test)
y_pred_prob_lgm_cc_ada= lgm_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_cc_ada, labels= lgm_cc_ada.classes_)
print(classification_report(y_test, y_pred_lgm_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_cc_ada))

[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.088320 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2633
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.96      0.04      0.08      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.13      9429
weighted avg       0.88      0.13      0.09      9429

Precision: 0.09907325684024713
Accuracy: 0.13246367589351998
AUC: 0.5988162992899649


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_cc_ada= GridSearchCV(estimator= lgm_cc_ada, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_cc_ada.fit(X_cc_ada, y_cc_ada)

print("Best Parameters:", gs_lgm_cc_ada.best_params_)
print("Best Precision Score:", gs_lgm_cc_ada.best_score_)

tuned_lgm_cc_ada= gs_lgm_cc_ada.best_estimator_
y_pred_tuned_lgm_cc_ada= tuned_lgm_cc_ada.predict(X_test)
y_pred_prob_tuned_lgm_cc_ada= tuned_lgm_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_cc_ada, labels= tuned_lgm_cc_ada.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_cc_ada))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.091603 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2633
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
              precision    recall  f1-score   support

           0       0.94      0.05      0.09      8517
           1       0.10      0.97      0.18       912

    accuracy                           0.14      9429
   macro avg       0.52      0.51      0.13      9429
weighted avg       0.86      0.14      0.10      9429

Precision: 0.09838799332962757
Accuracy: 0.13702407466327288
AUC: 0.5949180071230089


--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_cc_ada= CatBoostClassifier(random_state=15)
cat_cc_ada.fit(X_cc_ada, y_cc_ada)
y_pred_cat_cc_ada= cat_cc_ada.predict(X_test)
y_pred_prob_cat_cc_ada= cat_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_cc_ada, labels= cat_cc_ada.classes_)
print(classification_report(y_test, y_pred_cat_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_cc_ada))

Learning rate set to 0.024073
0:	learn: 0.6568304	total: 202ms	remaining: 3m 21s
1:	learn: 0.6181926	total: 216ms	remaining: 1m 47s
2:	learn: 0.5834684	total: 229ms	remaining: 1m 16s
3:	learn: 0.5500287	total: 239ms	remaining: 59.4s
4:	learn: 0.5198444	total: 250ms	remaining: 49.7s
5:	learn: 0.4904748	total: 270ms	remaining: 44.7s
6:	learn: 0.4653928	total: 290ms	remaining: 41.1s
7:	learn: 0.4459950	total: 300ms	remaining: 37.2s
8:	learn: 0.4265375	total: 315ms	remaining: 34.6s
9:	learn: 0.4062279	total: 325ms	remaining: 32.2s
10:	learn: 0.3889903	total: 337ms	remaining: 30.3s
11:	learn: 0.3718972	total: 347ms	remaining: 28.6s
12:	learn: 0.3585993	total: 359ms	remaining: 27.2s
13:	learn: 0.3453082	total: 372ms	remaining: 26.2s
14:	learn: 0.3344733	total: 387ms	remaining: 25.4s
15:	learn: 0.3221171	total: 398ms	remaining: 24.5s
16:	learn: 0.3111820	total: 411ms	remaining: 23.8s
17:	learn: 0.3015738	total: 426ms	remaining: 23.2s
18:	learn: 0.2911971	total: 436ms	remaining: 22.5s
19:	lear

168:	learn: 0.0843689	total: 3.16s	remaining: 15.5s
169:	learn: 0.0841895	total: 3.19s	remaining: 15.6s
170:	learn: 0.0837819	total: 3.2s	remaining: 15.5s
171:	learn: 0.0836161	total: 3.22s	remaining: 15.5s
172:	learn: 0.0834362	total: 3.25s	remaining: 15.5s
173:	learn: 0.0831364	total: 3.26s	remaining: 15.5s
174:	learn: 0.0829768	total: 3.28s	remaining: 15.5s
175:	learn: 0.0827726	total: 3.3s	remaining: 15.5s
176:	learn: 0.0825486	total: 3.32s	remaining: 15.4s
177:	learn: 0.0823973	total: 3.34s	remaining: 15.4s
178:	learn: 0.0822670	total: 3.36s	remaining: 15.4s
179:	learn: 0.0820435	total: 3.38s	remaining: 15.4s
180:	learn: 0.0818630	total: 3.4s	remaining: 15.4s
181:	learn: 0.0816977	total: 3.42s	remaining: 15.4s
182:	learn: 0.0815519	total: 3.43s	remaining: 15.3s
183:	learn: 0.0814127	total: 3.45s	remaining: 15.3s
184:	learn: 0.0812000	total: 3.47s	remaining: 15.3s
185:	learn: 0.0810577	total: 3.49s	remaining: 15.3s
186:	learn: 0.0808548	total: 3.51s	remaining: 15.2s
187:	learn: 0.0

334:	learn: 0.0591261	total: 6.14s	remaining: 12.2s
335:	learn: 0.0590316	total: 6.16s	remaining: 12.2s
336:	learn: 0.0589608	total: 6.18s	remaining: 12.2s
337:	learn: 0.0588861	total: 6.2s	remaining: 12.1s
338:	learn: 0.0587683	total: 6.22s	remaining: 12.1s
339:	learn: 0.0586396	total: 6.24s	remaining: 12.1s
340:	learn: 0.0584120	total: 6.26s	remaining: 12.1s
341:	learn: 0.0583258	total: 6.28s	remaining: 12.1s
342:	learn: 0.0582243	total: 6.3s	remaining: 12.1s
343:	learn: 0.0581676	total: 6.32s	remaining: 12s
344:	learn: 0.0580222	total: 6.33s	remaining: 12s
345:	learn: 0.0579516	total: 6.35s	remaining: 12s
346:	learn: 0.0578750	total: 6.36s	remaining: 12s
347:	learn: 0.0577964	total: 6.38s	remaining: 12s
348:	learn: 0.0576898	total: 6.41s	remaining: 12s
349:	learn: 0.0574883	total: 6.43s	remaining: 11.9s
350:	learn: 0.0572784	total: 6.46s	remaining: 11.9s
351:	learn: 0.0571811	total: 6.48s	remaining: 11.9s
352:	learn: 0.0570903	total: 6.5s	remaining: 11.9s
353:	learn: 0.0570264	total

504:	learn: 0.0436714	total: 8.88s	remaining: 8.7s
505:	learn: 0.0436170	total: 8.9s	remaining: 8.69s
506:	learn: 0.0435489	total: 8.91s	remaining: 8.67s
507:	learn: 0.0434989	total: 8.93s	remaining: 8.65s
508:	learn: 0.0434339	total: 8.95s	remaining: 8.63s
509:	learn: 0.0433853	total: 8.96s	remaining: 8.61s
510:	learn: 0.0433438	total: 8.98s	remaining: 8.59s
511:	learn: 0.0431759	total: 9.01s	remaining: 8.59s
512:	learn: 0.0431111	total: 9.03s	remaining: 8.57s
513:	learn: 0.0430136	total: 9.05s	remaining: 8.55s
514:	learn: 0.0429513	total: 9.07s	remaining: 8.54s
515:	learn: 0.0429022	total: 9.08s	remaining: 8.52s
516:	learn: 0.0427773	total: 9.1s	remaining: 8.5s
517:	learn: 0.0426394	total: 9.11s	remaining: 8.48s
518:	learn: 0.0425512	total: 9.13s	remaining: 8.46s
519:	learn: 0.0424939	total: 9.16s	remaining: 8.45s
520:	learn: 0.0424187	total: 9.19s	remaining: 8.45s
521:	learn: 0.0423748	total: 9.2s	remaining: 8.43s
522:	learn: 0.0423166	total: 9.22s	remaining: 8.41s
523:	learn: 0.042

665:	learn: 0.0337928	total: 11.4s	remaining: 5.72s
666:	learn: 0.0337434	total: 11.4s	remaining: 5.71s
667:	learn: 0.0336888	total: 11.4s	remaining: 5.69s
668:	learn: 0.0335875	total: 11.5s	remaining: 5.67s
669:	learn: 0.0335483	total: 11.5s	remaining: 5.66s
670:	learn: 0.0334629	total: 11.5s	remaining: 5.65s
671:	learn: 0.0334449	total: 11.5s	remaining: 5.63s
672:	learn: 0.0334179	total: 11.6s	remaining: 5.61s
673:	learn: 0.0333774	total: 11.6s	remaining: 5.6s
674:	learn: 0.0333068	total: 11.6s	remaining: 5.59s
675:	learn: 0.0332810	total: 11.6s	remaining: 5.57s
676:	learn: 0.0331995	total: 11.6s	remaining: 5.55s
677:	learn: 0.0331371	total: 11.7s	remaining: 5.54s
678:	learn: 0.0330825	total: 11.7s	remaining: 5.52s
679:	learn: 0.0330634	total: 11.7s	remaining: 5.5s
680:	learn: 0.0330183	total: 11.7s	remaining: 5.49s
681:	learn: 0.0329794	total: 11.7s	remaining: 5.47s
682:	learn: 0.0329177	total: 11.8s	remaining: 5.46s
683:	learn: 0.0328937	total: 11.8s	remaining: 5.44s
684:	learn: 0.

830:	learn: 0.0266544	total: 14.3s	remaining: 2.9s
831:	learn: 0.0266063	total: 14.3s	remaining: 2.88s
832:	learn: 0.0265624	total: 14.3s	remaining: 2.87s
833:	learn: 0.0265250	total: 14.3s	remaining: 2.85s
834:	learn: 0.0265182	total: 14.3s	remaining: 2.83s
835:	learn: 0.0265031	total: 14.3s	remaining: 2.81s
836:	learn: 0.0264862	total: 14.3s	remaining: 2.79s
837:	learn: 0.0264362	total: 14.4s	remaining: 2.77s
838:	learn: 0.0264187	total: 14.4s	remaining: 2.76s
839:	learn: 0.0263723	total: 14.4s	remaining: 2.74s
840:	learn: 0.0263422	total: 14.4s	remaining: 2.72s
841:	learn: 0.0263182	total: 14.4s	remaining: 2.7s
842:	learn: 0.0262900	total: 14.4s	remaining: 2.69s
843:	learn: 0.0262633	total: 14.4s	remaining: 2.67s
844:	learn: 0.0261976	total: 14.5s	remaining: 2.65s
845:	learn: 0.0261679	total: 14.5s	remaining: 2.63s
846:	learn: 0.0261359	total: 14.5s	remaining: 2.62s
847:	learn: 0.0260864	total: 14.5s	remaining: 2.6s
848:	learn: 0.0260554	total: 14.5s	remaining: 2.58s
849:	learn: 0.0

999:	learn: 0.0213891	total: 17.3s	remaining: 0us
              precision    recall  f1-score   support

           0       0.96      0.03      0.07      8517
           1       0.10      0.99      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.12      9429
weighted avg       0.88      0.13      0.08      9429

Precision: 0.0986856516976999
Accuracy: 0.12610032877293456
AUC: 0.6487374193820821


In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_cc_ada= GridSearchCV(estimator= cat_cc_ada, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_cc_ada.fit(X_cc_ada, y_cc_ada)

print("Best Parameters:", gs_cat_cc_ada.best_params_)
print("Best Precision Score:", gs_cat_cc_ada.best_score_)

tuned_cat_cc_ada= gs_cat_cc_ada.best_estimator_
y_pred_tuned_cat_cc_ada= tuned_cat_cc_ada.predict(X_test)
y_pred_prob_tuned_cat_cc_ada= tuned_cat_cc_ada.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_cc_ada, labels= tuned_cat_cc_ada.classes_)
print(classification_report(y_test, y_pred_tuned_cat_cc_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_cc_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_cc_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_cc_ada))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.3850807	total: 8.98ms	remaining: 889ms
1:	learn: 0.2387661	total: 18.7ms	remaining: 915ms
2:	learn: 0.1966672	total: 25.7ms	remaining: 832ms
3:	learn: 0.1710387	total: 38.9ms	remaining: 933ms
4:	learn: 0.1448033	total: 46.5ms	remaining: 884ms
5:	learn: 0.1335878	total: 52.7ms	remaining: 825ms
6:	learn: 0.1238633	total: 70.3ms	remaining: 934ms
7:	learn: 0.1156767	total: 78.9ms	remaining: 907ms
8:	learn: 0.1102261	total: 85.2ms	remaining: 861ms
9:	learn: 0.1033935	total: 93.1ms	remaining: 838ms
10:	learn: 0.0997939	total: 99.3ms	remaining: 804ms
11:	learn: 0.0960623	total: 108ms	remaining: 794ms
12:	learn: 0.0915214	total: 117ms	remaining: 782ms
13:	learn: 0.0893615	total: 124ms	remaining: 759ms
14:	learn: 0.0872758	total: 131ms	remaining: 745ms
15:	learn: 0.0849249	total: 138ms	remaining: 726ms
16:	learn: 0.0814248	total: 162ms	remaining: 789ms
17:	learn: 0.0788540	total: 169ms	remaining: 771ms
18:	learn: 0.076521

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_cc_ada, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_cc_ada, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_cc_ada, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_cc_ada, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_cc_ada, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_cc_ada, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_cc_ada, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_cc_ada, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_cc_ada, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_cc_ada, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_cc_ada, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_cc_ada, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_cc_ada, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_cc_ada, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_cc_ada, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_cc_ada, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_cc_ada, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_cc_ada, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_cc_ada, zero_division= 0))

Logistic Regression Precision: 0.2316919191919192
Decision Tree Precision: 0.10491044230534909
Tuned Decision Tree Precision: 0.10300939693112882
KNeighborsClassifier Precision: 0.22422535211267605
Tuned KNeighborsClassifier Precision: 0.18124578367438723
GaussianNB Precision: 0.13855622089155023
SVM Precision: 0.1267056530214425
Random Forest Precision: 0.10107861670187925
Tuned Random Forest Precision: 0.10183938986092418
AdaBoost Precision: 0.09903729113643908
Tuned AdaBoost Precision: 0.09790895229797429
GradientBoosting Precision: 0.09981167608286252
Tuned GradientBoosting Precision: 0.09869216397406308
XGB Precision: 0.0981169474727453
Tuned XGB Precision: 0.09855264611645122
LGBM Precision: 0.09907325684024713
Tuned LGBM Precision: 0.09838799332962757
CatBoost Precision: 0.0986856516976999
Tuned CatBoost Precision: 0.09774933804060018


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_cc_ada, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_cc_ada, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_cc_ada, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_cc_ada, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_cc_ada, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_cc_ada, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_cc_ada, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_cc_ada, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_cc_ada, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_cc_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_cc_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_cc_ada, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_cc_ada, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_cc_ada, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_cc_ada, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_cc_ada, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_cc_ada, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_cc_ada, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_cc_ada, zero_division= 0)
                  }

cc_ada_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
cc_ada_precision= cc_ada_precision.sort_values(by= 'Precision Score', ascending=False)
print(cc_ada_precision)

                                    Model  Precision Score
0          Logistic Regression Precision:         0.231692
3         KNeighborsClassifier Precision:         0.224225
4   Tuned KNeighborsClassifier Precision:         0.181246
5                   GaussianNB Precision:         0.138556
6                          SVM Precision:         0.126706
1                Decision Tree Precision:         0.104910
2          Tuned Decision Tree Precision:         0.103009
8          Tuned Random Forest Precision:         0.101839
7                Random Forest Precision:         0.101079
11            GradientBoosting Precision:         0.099812
15                        LGBM Precision:         0.099073
9                     AdaBoost Precision:         0.099037
12      Tuned GradientBoosting Precision:         0.098692
17                    CatBoost Precision:         0.098686
14                   Tuned XGB Precision:         0.098553
16                  Tuned LGBM Precision:         0.0983

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_cc_ada),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_cc_ada),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_cc_ada),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_cc_ada),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_cc_ada),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_cc_ada),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_cc_ada),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_cc_ada),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_cc_ada),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_cc_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_cc_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_cc_ada),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_cc_ada),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_cc_ada),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_cc_ada),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_cc_ada),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_cc_ada),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_cc_ada),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_cc_ada)
                  }

cc_ada_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
cc_ada_accuracy= cc_ada_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(cc_ada_accuracy)

                                   Model  Accuracy Score
6                          SVM Accuracy:        0.862658
3         KNeighborsClassifier Accuracy:        0.799449
0          Logistic Regression Accuracy:        0.722982
4   Tuned KNeighborsClassifier Accuracy:        0.602609
5                   GaussianNB Accuracy:        0.442359
1                Decision Tree Accuracy:        0.215505
2          Tuned Decision Tree Accuracy:        0.195355
8          Tuned Random Forest Accuracy:        0.150281
7                Random Forest Accuracy:        0.142327
11            GradientBoosting Accuracy:        0.137024
16                  Tuned LGBM Accuracy:        0.137024
9                     AdaBoost Accuracy:        0.134691
14                   Tuned XGB Accuracy:        0.132570
15                        LGBM Accuracy:        0.132464
18              Tuned CatBoost Accuracy:        0.129918
13                         XGB Accuracy:        0.129176
12      Tuned GradientBoosting 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_cc_ada),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_cc_ada),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_cc_ada),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_cc_ada),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_cc_ada),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_cc_ada),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_cc_ada),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_cc_ada),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_cc_ada),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_cc_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_cc_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_cc_ada),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_cc_ada),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_cc_ada),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_cc_ada),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_cc_ada),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_cc_ada),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_cc_ada),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_cc_ada)
                  }

cc_ada_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
cc_ada_auc= cc_ada_auc.sort_values(by= 'AUC Score', ascending=False)
print(cc_ada_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.830842
4   Tuned KNeighborsClassifier AUC:   0.798561
5                   GaussianNB AUC:   0.794083
3         KNeighborsClassifier AUC:   0.736064
9                     AdaBoost AUC:   0.665672
11            GradientBoosting AUC:   0.662993
8          Tuned Random Forest AUC:   0.652597
12      Tuned GradientBoosting AUC:   0.651657
7                Random Forest AUC:   0.650165
17                    CatBoost AUC:   0.648737
10              Tuned AdaBoost AUC:   0.633295
13                         XGB AUC:   0.626864
14                   Tuned XGB AUC:   0.609627
18              Tuned CatBoost AUC:   0.606113
15                        LGBM AUC:   0.598816
16                  Tuned LGBM AUC:   0.594918
6                          SVM AUC:   0.588696
2          Tuned Decision Tree AUC:   0.541526
1                Decision Tree AUC:   0.540784
